<a href="https://colab.research.google.com/github/neilc-itv/itv-geo-r/blob/main/Python/lookup-google-analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Make sure you have set up the Google Analytics Data API before running this code
#Make sure you have installed the library google-analytics-data ( pip install google-analytics-data)

from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange
from google.analytics.data_v1beta.types import Dimension
from google.analytics.data_v1beta.types import Metric
from google.analytics.data_v1beta.types import RunReportRequest
import pandas as pd
import requests
import io
import os

In [ ]:
# replace service_account_file_path with the value of your JSON file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_file_path

In [ ]:
def sample_run_report(property_id, export_path):
    """Runs a simple report on a Google Analytics 4 property."""

    client = BetaAnalyticsDataClient()

    # Replace Property with the ID of your property.
    # To find your Property ID, navigate to the Google Analytics Admin setting and go Property Settings.
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name="city")],
        metrics=[Metric(name="activeUsers")],
        date_ranges=[DateRange(start_date="2022-09-01", end_date="today")],
    )
    response = client.run_report(request)v

    output = []
    print("Report result:")
    for row in response.rows:
      output.append({"City":row.dimension_values[0].value, "Sessions": row.metric_values[0].value})
    df = pd.DataFrame(output)
    df.to_csv(export_path)
  
sample_run_report(PROPERTY_ID, "export.csv")
gadata = pd.read_csv("export.csv")
gadata.head(5)

In [ ]:
# Download the Region Lookup csv file from our GitHub account

url = "https://raw.githubusercontent.com/neilc-itv/itv-geo-r/main/Python/ITV%20Regions%20Mapping%20Tables%20-%20GA%20Lookup.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

region_lookup = pd.read_csv(io.StringIO(download.decode('utf-8')))
web_traffic_regional = gadata.merge(region_lookup, how='left', left_on='City', right_on='City')
web_traffic_regional.head(10)